## Обработка отсутствующих данных

In [1]:
import numpy as np
import pandas as pd

##### В библиотеке Numpy и Pandas значения NaN None и null сводятся к значению NaN, которое существует не для всех типов данных, а только для float64, соответстенно для типов int автоматически происходит повышающее преобразование типов к float64.
    - в качестве компромиссов могут быть использованы 2 основных стратегии:
        1) использование маски, отмечающей глобально отсутствующие значения (ест доп память)
        2) выбор специального значения индикатора (sentinel value), обозначающего пропущеное значение, в качестве значения индикатора может быть выбрано какое-то редкое значение или комбинация битов (например -9999) (сокращает доступный диапазон значение, и требуе дополнительной неоптимизированной логики при арифметических операциях)

In [3]:
rng = np.random.RandomState(42)

In [6]:
A = pd.DataFrame(rng.randint(0, 20, (2,2)),
                columns=list('AB'))

In [7]:
B = pd.DataFrame(rng.randint(0, 10, (3,3)),
                columns=list('BAC'))

In [8]:
A

,A,B
0,10,3
1,7,2


In [9]:
B

,B,A,C
0,5,4,1
1,7,5,1
2,4,0,9


In [10]:
A.dtypes

A    int32
B    int32
dtype: object

In [13]:
B.dtypes

B    int32
A    int32
C    int32
dtype: object

In [14]:
C = A + B

In [15]:
C

,A,B,C
0,14.0,8.0,NaN
1,12.0,9.0,NaN
2,NaN,NaN,NaN


In [16]:
C.dtypes

A    float64
B    float64
C    float64
dtype: object

In [17]:
# для типов int автоматически происходит повышающее преобразование типов к float64 при встрече с NaN
x = pd.Series(range(5), dtype=int)

In [18]:
x

0    0
1    1
2    2
3    3
4    4
dtype: int32

In [19]:
x[0] = None

In [20]:
x

0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [22]:
# строковые данные всегда хранятся в Pandas с типом Object 
a = np.array(['a', 'b', 'c', 'd'])

In [23]:
a

array(['a', 'b', 'c', 'd'], dtype='<U1')

In [24]:
a.dtype

dtype('<U1')

In [25]:
str_a = pd.DataFrame(a)

In [28]:
str_a.dtypes

0    object
dtype: object

In [ ]:
# np.nan

In [34]:
vals = np.array([1, np.nan, 3])

In [35]:
vals

array([ 1., nan,  3.])

In [31]:
vals.dtype

dtype('float64')

##### Правила повышающего поведения типов Pandas:
##### Класс типов                               Преобразование при хранении NA               Значение-индекатор NA
    С плавающей точкой                   Без изменений                          np.nan
    Object                               Без изменений                     None или np.nan
    Целое число                        Приводится к float64                     np.nan
    Булево значение                    Приводится к Object                 None или np.nan

Значение NaN является вирусным, оно превращает в NaN любое значение с которым провзаимодействует, соответственно любые агрегации с данными в которых есть NaN так же дадут NaN
##### для работы с агрегациями есть спец ф-ии: np.nansum(val), np.nanmin(val), np.nanmax(val)

In [36]:
vals.sum()

nan

In [37]:
np.nansum(vals)

4.0

### Операции над пустыми значениями
    isnull()  - генерирует булеву маску для отсутствующих значений
    notnull() - противоволожность isnull()
    dropna()  - отбрасывает NaN
    fillna()  - возвращает копию данных, в которой пропущенные значения заполнены или восстановлены

#### Выявление пустых значений:

In [38]:
data = pd.Series([1, np.nan, 'hello', None])

In [39]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [40]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [41]:
data.isna()

0    False
1     True
2    False
3     True
dtype: bool

In [42]:
# использование булевой маски
data[data.notnull()]

0        1
2    hello
dtype: object

#### Удаление пустых значений:

In [43]:
data.dropna()

0        1
2    hello
dtype: object

In [44]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [46]:
df = pd.DataFrame([
    [1, np.nan, 2],
    [2, 3, 5],
    [np.nan, 4, 6]
])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [47]:
df.dropna()
# из DataFrame можно выбросить только целые строки или столбцы

,0,1,2
1,2.0,3.0,5


In [48]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [ ]:
# доп параметры how и thresh
# how = 'any' отбрасываются все строки/столбцы где есть хоть одно пустое знач.
# how = 'all' только те, где все пустые
# thresh = 3 только те, где пустых знач. больше 3

#### Заполнение пустых значений:

In [49]:
df.fillna(0)

,0,1,2
0,1.0,0.0,2
1,2.0,3.0,5
2,0.0,4.0,6


In [50]:
# заполнение через копирование предыдущих значений (вперёд)
df.fillna(method='ffill')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,4.0,6


In [51]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [52]:
# заполнение через копирование назад
df.fillna(method='bfill')

,0,1,2
0,1.0,3.0,2
1,2.0,3.0,5
2,NaN,4.0,6


In [53]:
# можно задать ось
df.fillna(method='bfill', axis=1)

,0,1,2
0,1.0,2.0,2.0
1,2.0,3.0,5.0
2,4.0,4.0,6.0
